In [ ]:
# !pip install tensorflow
!pip install keras-tuner
!pip install opendatasets

In [ ]:
import tensorflow
from tensorflow import keras
import keras_tuner as kt
from keras.layers import Dense, Dropout
from keras.models import Sequential
import matplotlib.pyplot as plt
import opendatasets as od
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
od.download("https://www.kaggle.com/datasets/muhamedumarjamil/student-placement-data-with-cgpa-and-salary")
data=pd.read_csv("/content/student-placement-data-with-cgpa-and-salary/Placement.csv")

Skipping, found downloaded files in "./student-placement-data-with-cgpa-and-salary" (use force=True to force download)


In [ ]:
data.drop(columns=["Salary (INR LPA)", "Student_ID"], inplace=True)


In [ ]:
data.head(5)

,CGPA,Internships,Placed
0,7.90,3,Yes
1,7.39,0,Yes
2,8.02,2,Yes
3,8.72,4,Yes
4,7.31,2,Yes


In [ ]:
st=StandardScaler()
le=LabelEncoder()
data["CGPA"]=st.fit_transform(data[["CGPA"]])
data["Internships"]=st.fit_transform(data[["Internships"]])
data["Placed"] = le.fit_transform(data["Placed"])

In [ ]:
data.head(5)

,CGPA,Internships,Placed
0,0.490964,0.670779,1
1,-0.160351,-1.436379,1
2,0.644215,-0.031607,1
3,1.538178,1.373164,1
4,-0.262519,-0.031607,1


In [ ]:
x_train, x_test, y_train, y_test=train_test_split(data.drop(columns=["Placed"]), data["Placed"], test_size=0.2, random_state=2)

In [ ]:
class my_keras_tuner:

  # def build(self, hp):
  #   model = Sequential()
  #   model.add(Dense(32, activation="relu", input_dim=x_train.shape[1]))
  #   model.add(Dense(1, activation="sigmoid"))
  #   optimizer = hp.choice("optimizer", values=self.opt)
  #   model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])
  #   return model

  # def declare_hyperparameters(self, hp):
  #   hp.choice("optimizer", values=self.opt)
  def get_optimizer(self, opt):
    self.opt=opt

  def get_best_layers(self):
    pass

  def get_best_activation(self):
    pass

  def get_best_optimizer(self):
    def build_model(hp):
          model = Sequential()
          model.add(Dense(128, activation='relu'))
          model.add(Dropout(0.2))
          model.add(Dense(1, activation='sigmoid'))
          hp_optimizer = hp.Choice('optimizer', values=self.opt)
          # hp_learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

          # Compile the model
          model.compile(
              optimizer=hp_optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
          )

          # Set the learning rate on the chosen optimizer
          model.optimizer.learning_rate = 0.1

          return model
    tuner = kt.RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=8,  # Fewer trials needed as we are tuning fewer HPs
        executions_per_trial=2,
        directory='optimizer_only_dir',
        project_name='tune_optimizer'
    )

    # 4. Run the search
    print("Starting search for the best optimizer...")
    tuner.search(
        x_train,
        y_train,
        epochs=15,
        validation_split=0.2
    )
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    print(f"Search complete! The best optimizer is '{best_hps.get('optimizer')}'")

  def get_best_nodes(self):
    pass

  def check(self):
    print("the input is: ", self.opt)

In [ ]:
mkt=my_keras_tuner()

testing for optimizer

In [ ]:
values=['adam', 'sgd', 'rmsprop']
mkt.get_optimizer(values)
mkt.get_best_optimizer()

Reloading Tuner from optimizer_only_dir/tune_optimizer/tuner0.json
Starting search for the best optimizer...
Search complete! The best optimizer is 'sgd'


In [ ]:
def making_model(layers, nodes):
  model=Sequential()
  for i, nodes in enumerate(nodes):
    if i==0:
      model.add(Dense(nodes, activation="relu", input_dim=2))
    else:
      model.add(Dense(nodes, activation="relu"))
  model.add(Dense(1, activation="sigmoid"))
  model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
  return model

model = making_model(layers=2, nodes=[128,32])
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 128)            │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,545 (17.75 KB)

 Trainable params: 4,545 (17.75 KB)

 Non-trainable params: 0 (0.00 B)